In [1]:
import os

def read_data(filename):
    with open(filename, 'r', encoding="cp949") as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data

data = read_data(r'C:\Users\HANA\PycharmProjects\HANATOUR\NLP\TEXT_Example\ratings_morphed.txt')
data_text = [line[1] for line in data]
data_senti = [line[2] for line in data]

In [4]:
from sklearn.model_selection import train_test_split

train_data_text, test_data_text, train_data_senti, test_data_senti \
    = train_test_split(
        data_text,
        data_senti,
        stratify=data_senti,
        random_state=156
)

In [5]:
# Counter 클래스를 이용해 각 분류가 같은 비율로 들어갔는지 확인해 본다
from collections import Counter
train_data_senti_freq = Counter(train_data_senti)
print('train_data_senti_freq:', train_data_senti_freq)
test_data_senti_freq = Counter(test_data_senti)
print('test_data_senti_freq:', test_data_senti_freq)
#예제데이타를 읽어서 테스트와 트레인을 나눈다.

train_data_senti_freq: Counter({'1': 74097, '0': 74058})
test_data_senti_freq: Counter({'1': 24700, '0': 24686})


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=5, ngram_range=(1,2)).fit(train_data_text)
X_train = vect.transform(train_data_text)

print("X_train:\n", repr(X_train))
feature_names = vect.get_feature_names()
print("특성 개수:", len(feature_names))
print("처음 20개 특성:\m", feature_names[:20])
print("3000~5000까지의 특성:\n", feature_names[3000:5000])

X_train:
 <148155x28571 sparse matrix of type '<class 'numpy.int64'>'
	with 1086129 stored elements in Compressed Sparse Row format>
특성 개수: 28571
처음 20개 특성:\m ['10점', '10점 10점', '10점 ㅋㅋ', '10점 ㅎㅎ', '10점 그냥', '10점 그래도', '10점 기대', '10점 내용', '10점 너무', '10점 누르', '10점 드리', '10점 드림', '10점 만들', '10점 만점', '10점 매미', '10점 모자', '10점 모자라', '10점 배우', '10점 부족', '10점 사람']
3000~5000까지의 특성:
 ['극우', '극의', '극의 흐름', '극장', '극장 ㅋㅋ', '극장 ㅜㅜ', '극장 감동', '극장 개봉', '극장 관객', '극장 관람', '극장 기억', '극장 나오', '극장 너무', '극장 다시', '극장 못하', '극장 본사', '극장 사람', '극장 상영', '극장 아깝', '극장 아쉽', '극장 아직', '극장 영화', '극장 재미있', '극장 정말', '극장 중간', '극장 지금', '극장 진짜', '극장 처음', '극장 혼자', '극장 후회', '극장가', '극장가 사람', '극장가 영화', '극장판', '극장판 가장', '극장판 나오', '극장판 너무', '극장판 만들', '극장판 시리즈', '극장판 애니', '극장판 재미있', '극장판 제일', '극장판 최고', '극장판 최악', '극적', '극적 요소', '극찬', '극찬 영화', '극초', '극치', '극치 달리', '극치 보이', '극한', '극한 상황', '극화', '극히', '근거', '근대', '근대사', '근데', '근데 ㅋㅋ', '근데 감독', '근데 결말', '근데 남자', '근데 내용', '근데 너무', '근데 네이버', '근데 마지막', '근데 배우', '근데 사람', '근데 솔직히', '근데 스토리',

In [8]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
y_train = pd.Series(train_data_senti)
scores = cross_val_score(LogisticRegression(solver="liblinear", C=1), X_train, y_train, cv=5)
print('교차 검증 점수:', scores)
print('교차 검증 점수 평균:', scores.mean())

from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.01, 0.1, 1, 3, 5]}
grid = GridSearchCV(LogisticRegression(solver="liblinear"), param_grid, cv=5)
grid.fit(X_train, y_train)
print("최고 교차 검증 점수:", round(grid.best_score_, 3))
print("최적의 매개변수:", grid.best_params_)

X_test = vect.transform(test_data_text)
y_test = pd.Series(test_data_senti)
print("테스트 데이터 점수:", grid.score(X_test, y_test))

교차 검증 점수: [0.81623975 0.81553103 0.81313489 0.81424859 0.81542979]
교차 검증 점수 평균: 0.8149168100975329
최고 교차 검증 점수: 0.815
최적의 매개변수: {'C': 1}
테스트 데이터 점수: 0.8185923136111448


In [9]:
#리뷰계산을 위해서 벡터화 테스트삼아 TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(min_df=5, ngram_range=(1,2)).fit(train_data_text)
X_train = vect.transform(train_data_text)

print("X_train:\n", repr(X_train))
feature_names = vect.get_feature_names()
print("특성 개수:", len(feature_names))
print("처음 20개 특성:\m", feature_names[:20])
print("3000~5000까지의 특성:\n", feature_names[3000:5000])

import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
y_train = pd.Series(train_data_senti)
scores = cross_val_score(LogisticRegression(solver="liblinear", C=1), X_train, y_train, cv=5)
print('교차 검증 점수:', scores)
print('교차 검증 점수 평균:', scores.mean())

from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.01, 0.1, 1, 3, 5]}
grid = GridSearchCV(LogisticRegression(solver="liblinear"), param_grid, cv=5)
grid.fit(X_train, y_train)
print("최고 교차 검증 점수:", round(grid.best_score_, 3))
print("최적의 매개변수:", grid.best_params_)

X_test = vect.transform(test_data_text)
y_test = pd.Series(test_data_senti)
print("테스트 데이터 점수:", grid.score(X_test, y_test))

X_train:
 <148155x28571 sparse matrix of type '<class 'numpy.float64'>'
	with 1086129 stored elements in Compressed Sparse Row format>
특성 개수: 28571
처음 20개 특성:\m ['10점', '10점 10점', '10점 ㅋㅋ', '10점 ㅎㅎ', '10점 그냥', '10점 그래도', '10점 기대', '10점 내용', '10점 너무', '10점 누르', '10점 드리', '10점 드림', '10점 만들', '10점 만점', '10점 매미', '10점 모자', '10점 모자라', '10점 배우', '10점 부족', '10점 사람']
3000~5000까지의 특성:
 ['극우', '극의', '극의 흐름', '극장', '극장 ㅋㅋ', '극장 ㅜㅜ', '극장 감동', '극장 개봉', '극장 관객', '극장 관람', '극장 기억', '극장 나오', '극장 너무', '극장 다시', '극장 못하', '극장 본사', '극장 사람', '극장 상영', '극장 아깝', '극장 아쉽', '극장 아직', '극장 영화', '극장 재미있', '극장 정말', '극장 중간', '극장 지금', '극장 진짜', '극장 처음', '극장 혼자', '극장 후회', '극장가', '극장가 사람', '극장가 영화', '극장판', '극장판 가장', '극장판 나오', '극장판 너무', '극장판 만들', '극장판 시리즈', '극장판 애니', '극장판 재미있', '극장판 제일', '극장판 최고', '극장판 최악', '극적', '극적 요소', '극찬', '극찬 영화', '극초', '극치', '극치 달리', '극치 보이', '극한', '극한 상황', '극화', '극히', '근거', '근대', '근대사', '근데', '근데 ㅋㅋ', '근데 감독', '근데 결말', '근데 남자', '근데 내용', '근데 너무', '근데 네이버', '근데 마지막', '근데 배우', '근데 사람', '근데 솔직히', '근데 스토리

교차 검증 점수: [0.81762343 0.81866964 0.81910837 0.81775843 0.81991833]
교차 검증 점수 평균: 0.818615639026695
최고 교차 검증 점수: 0.819
최적의 매개변수: {'C': 1}
테스트 데이터 점수: 0.8208399141457093


In [12]:
import rhinoMorph
rn = rhinoMorph.startRhino()
print('rn\n',rn)
new_input = '오늘은 정말 재미있는 하루구나!'

# 입력 데이터 형태소 분석하기
inputdata = []
morphed_input = rhinoMorph.onlyMorph_list(rn, new_input, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
morphed_input = ' '.join(morphed_input)                     # 한 개의 문자열로 만들기
inputdata.append(morphed_input)                               # 분석 결과를 리스트로 만들기
X_input = vect.transform(inputdata)
print(float(grid.predict(X_input)))
result = grid.predict(X_input) # 0은 부정,1은 긍정
print(result)
print(type(result))
if result == 0.0:
    print("부정적인 글입니다")
else:
    print("긍정적인 글입니다")
    
    
new_input = '뷰방 예약했는데 이상한 다락방 줘서 첫날부터 기분 잡치고 다음날부터 좋은방 준다고해서 기대했더니 뷰방보다 좋지도 않음...'

# 입력 데이터 형태소 분석하기
inputdata = []
morphed_input = rhinoMorph.onlyMorph_list(rn, new_input, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
morphed_input = ' '.join(morphed_input)                     # 한 개의 문자열로 만들기
inputdata.append(morphed_input)                               # 분석 결과를 리스트로 만들기
X_input = vect.transform(inputdata)
print(float(grid.predict(X_input)))
result = grid.predict(X_input) # 0은 부정,1은 긍정
print(result)
print(type(result))
if result == 0.0:
    print("부정적인 글입니다")
else:
    print("긍정적인 글입니다")

filepath:  c:\users\hana\pycharmprojects\hanatour\venv\lib\site-packages
classpath:  c:\users\hana\pycharmprojects\hanatour\venv\lib\site-packages\rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!
rn
 <java class 'rhino.RHINO'>
1.0
['1']
<class 'numpy.ndarray'>
긍정적인 글입니다
0.0
['0']
<class 'numpy.ndarray'>
긍정적인 글입니다
